In [10]:
import os
from keras.optimizers import SGD
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from lib.data_loader import DataLoader
from lib.image import ImageDataGenerator
from lib.resnet_model import Resnet3DBuilder

Using TensorFlow backend.


In [2]:
root_directory = r'<path>\dataset'
labels_csv_path = os.path.join(root_directory, 'labels_extracted.csv')
train_csv_path = os.path.join(root_directory, 'train_extracted.csv')
val_csv_path = os.path.join(root_directory, 'validation_extracted.csv')
train_path = os.path.join(root_directory, 'Train')
val_path = os.path.join(root_directory, 'Validation')

data = DataLoader(labels_csv_path, train_csv_path, val_csv_path)

In [3]:
data.train_df

,video_id,label
0,169,Swiping Left
1,327,Swiping Left
2,429,Swiping Left
3,477,Swiping Left
4,579,Swiping Left
...,...,...
7995,63147,No gesture
7996,63224,No gesture
7997,63235,No gesture
7998,63246,No gesture


In [4]:
data.labels_df

,label
0,Swiping Left
1,Swiping Right
2,Swiping Down
3,Swiping Up
4,Sliding Two Fingers Down
5,Sliding Two Fingers Up
6,Thumb Down
7,Thumb Up
8,Stop Sign
9,No gesture


In [5]:
data.label_to_int

{'Swiping Left': 0,
 'Swiping Right': 1,
 'Swiping Down': 2,
 'Swiping Up': 3,
 'Sliding Two Fingers Down': 4,
 'Sliding Two Fingers Up': 5,
 'Thumb Down': 6,
 'Thumb Up': 7,
 'Stop Sign': 8,
 'No gesture': 9}

In [3]:
WIDTH = 96
HEIGHT = 64
TARGET_SIZE = (HEIGHT, WIDTH)
BATCH_SIZE = 32
N_FRAMES = 16
SKIP = 1
INPUT_SHAPE = (N_FRAMES,) + TARGET_SIZE + (3,)
N_CLASSES = 10
DROP_RATE = 0.5
LEARNING_RATE = 0.01
EPOCHS = 10
WORKERS = 2

In [7]:
model = Resnet3DBuilder.build_resnet_101(
    input_shape=INPUT_SHAPE, 
    num_outputs=N_CLASSES,
    drop_rate=DROP_RATE
)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16, 64, 96, 3 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 8, 32, 48, 64 65920       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 8, 32, 48, 64 256         conv3d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 8, 32, 48, 64 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [8]:
optimizer = SGD(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
def train_model(model):

    image_datagen = ImageDataGenerator()

    train_generator = image_datagen.flow_video_from_dataframe(
        dataframe=data.train_df, 
        directory=train_path, 
        path_classes=labels_csv_path, 
        x_col='video_id', 
        y_col='label', 
        target_size=TARGET_SIZE, 
        class_mode='categorical',
        color_mode='rgb',
        batch_size=BATCH_SIZE, 
        shuffle=True,
        seed=42,
        nb_frames=N_FRAMES, 
        skip=SKIP
    )

    validation_generator = image_datagen.flow_video_from_dataframe(
        dataframe=data.val_df, 
        directory=val_path, 
        path_classes=labels_csv_path, 
        x_col='video_id', 
        y_col='label', 
        target_size=TARGET_SIZE, 
        class_mode='categorical',
        color_mode='rgb',
        batch_size=BATCH_SIZE, 
        shuffle=True,
        seed=42,
        nb_frames=N_FRAMES, 
        skip=SKIP
    )

    n_sample_train = len(data.train_df)
    n_sample_val = len(data.val_df)

    # model_checkpoint = ModelCheckpoint(
    #     'output/models/resnet_101_{epoch:02d}.h5', 
    #     period=1, 
    #     save_weights_only=False
    # )

    history = model.fit_generator(
        generator=train_generator, 
        steps_per_epoch=n_sample_train/BATCH_SIZE, 
        validation_data=validation_generator, 
        validation_steps=n_sample_val/BATCH_SIZE,
        epochs=1, 
        workers=WORKERS,
        verbose=1,
        # callbacks=[model_checkpoint]
    )

    model.save('output/models/resnet_101.h5')

    return history

In [1]:
# epoch 1
history = train_model(model)

Found 8000 video folders belonging to 10 classes.
Found 2000 video folders belonging to 10 classes.
Epoch 1/1
250/250 [==============================] - 8806s 35s/step - loss: 12.8543 - accuracy: 0.1330 - val_loss: 13.0108 - val_accuracy: 0.1390


In [2]:
# epoch 2
model = load_model('output/models/resnet_101_1.h5')
history = train_model(model)

Found 8000 video folders belonging to 10 classes.
Found 2000 video folders belonging to 10 classes.
Epoch 1/1
250/250 [==============================] - 18130s 73s/step - loss: 12.4795 - accuracy: 0.3038 - val_loss: 12.7454 - val_accuracy: 0.3715


In [3]:
# epoch 3
model = load_model('output/models/resnet_101_2.h5')
history = train_model(model)

Found 8000 video folders belonging to 10 classes.
Found 2000 video folders belonging to 10 classes.
Epoch 1/1
250/250 [==============================] - 7889s 32s/step - loss: 12.0474 - accuracy: 0.4710 - val_loss: 11.9053 - val_accuracy: 0.4425


In [1]:
# epoch 4 - 5
model = load_model('output/models/resnet_101_3.h5')
history = train_model(model)

Found 8000 video folders belonging to 10 classes.
Found 2000 video folders belonging to 10 classes.
Epoch 1/2
250/250 [==============================] - 8987s 36s/step - loss: 11.7269 - accuracy: 0.5990 - val_loss: 11.7562 - val_accuracy: 0.5835
Epoch 2/2
250/250 [==============================] - 11044s 44s/step - loss: 11.5250 - accuracy: 0.6630 - val_loss: 11.5108 - val_accuracy: 0.6745


In [9]:
# epoch 6 - 7
model = load_model('output/models/resnet_101_5.h5')
history = train_model(model)

Found 8000 video folders belonging to 10 classes.
Found 2000 video folders belonging to 10 classes.
Epoch 1/2
250/250 [==============================] - 11356s 45s/step - loss: 11.3243 - accuracy: 0.7379 - val_loss: 11.3258 - val_accuracy: 0.7305
Epoch 2/2
250/250 [==============================] - 11786s 47s/step - loss: 11.1146 - accuracy: 0.8251 - val_loss: 11.1201 - val_accuracy: 0.8198
